In [1]:
import pandas as pd # standard python data library
import geopandas as gp # the geo-version of pandas
import numpy as np 
import os
import fiona
from statistics import mean, median
from pandas import read_csv
gp.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw' #To load KML files
import string
import xml.etree.ElementTree as et

pd.options.display.max_columns = 999

# VEST CT 2018

## VEST Documentation

### Sources

#### Election Results
Election results from the Connecticut Secretary of State (https://portal.ct.gov/SOTS/Election-Services/Election-Results/Election-Results).

#### Shapefiles
Precinct shapefile from the U.S. Census Bureau's 2020 Redistricting Data Program.

### Processing
District splits not reported separately were merged for Bethel 5, Bridgeport 129-3, 130-2, Durham 3, East Haven 3-3, Hartford 11, 12, 24, Naugatuck 3-3, New Britain 12, New Haven 9-2, 17, 21-1, Stamford 3, 5, 6-1, 8, 12, 20-1, 21, Torrington 6, 7, Waterbury 74-5.

The Stratford and Westport precincts were renumbered from municipal districts to state/federal districts.

The following additional modifications were made to match the 2018 precinct boundaries.

Ansonia: Adjust 1/2 to match shapefile  
Avon: Adjust 1/3 to match street list  
Berlin: Adjust 2/5 to match voter file  
Bethel: Adjust 1/4 to match voter file  
Bridgeport: Adjust 124-2/3/4, 126-1/2/5, 130-3/4 to match PDF  
Colchester: Adjust 1/3, 2/4 to match street list  
Coventry: Adjust 1/2 to match street list  
Danbury: Align wards 4/5, 6/7 with PDF  
Darien: Align 1/5, 1/6, 2/4 with shapefile  
East Hartford: Adjust 1/2, 5/6 to match street list  
East Haven: Add 1-3, 5-3 to match street list; Align 3, 3-3 with LD  
East Windsor: Align 1, 1-2 with LD  
Enfield: Adjust 258/458 to match PDF  
Fairfield: Split 3-32/3-34 by LD; Adjust 8/9 to match PDF  
Glastonbury: Adjust 4/5, 4/9, 7/9 to match PDF  
Greenwich: Split 10/10-1 by LD; Adjust 1/2/3 to match PDF  
Guilford: Adjust 1/3, 2/3 to match descriptions  
Haddam: Adjust 1/2 to match street list  
Hamden: Adjust 1/9, 5/6 to match PDF  
Killingly: Adjust 2/4, 3/4 to match PDF   
Ledyard: Adjust 1/2 to match street list  
Litchfield: Adjust 2/4 to match voter file  
Manchester: Merge 5/9; Adjust 3/5 to match street list  
Middletown: Adjust 1/12 to match GIS  
Milford: Align 117/119, 119-1/3 with LD and PDF  
New Britain: Adjust 12/14 to match voter file  
New Haven: Split 11-1/11-3 and align VTDs with voter file  
New Milford: Align all VTDs with voter file  
Newtown: Adjust 1/2 to match street list  
Norwich: Adjust 4/5 to match voter file  
Plainfield: Adjust 1/3, 2/4 to match voter file  
Ridgefield: Adjust 1/2, 1/3 to match voter file  
Rocky Hill: Adjust 2/3 to match voter file  
Seymour: Adjust 1/3, 2/3 to match voter file  
Shelton: Adjust 1/4, 2/3 to match PDF, voter file  
Simsbury: Align 1/2 with 2017 redistricting  
South Windsor: Adjust 3/5, 4/5 to match PDF  
Southbury: Align all VTDs with shapefile, street list  
Southington: Adjust 3/6, 5/8, 8/9 to match PDF  
Stafford: Adjust 1/2 to match PDF  
Stamford: Merge 1/23 and adjust 6/7 to reverse 2019 redistricting  
Stonington: Align all VTDs with shapefile  
Stratford: Split 20-1/13, 80-1/21 by LD; Align VTDs with voter file  
Vernon: Adjust 1/2, 2/3 to match PDF  
Wallingford: Adjust 2/3, 2/4, 7/8 to match voter file  
Waterbury: Align VTDs with PDF, street list, voter file  
West Haven: Adjust 1/7, 4/5/6, 8/9/10 to match voter file  
Weston: Adjust 1/2 to match voter file  
Windham: Adjust 4/6 to match street list  
Windsor: Adjust 1/3 to match street list  
Wolcott: Adjust 1/2, 1/3 to match voter file  

### Races
G18USSDMUR - Christopher S. Murphy (Democratic Party and Working Families Party (fusion candidate))  
G18USSRCOR - Matthew Corey (Republican Party)  
G18USSLLIO - Richard Lion (Libertarian Party)  
G18USSGRUS - Jeff Russell (Green Party)  
G18USSOWRI - Write-in Votes  

G18GOVDLAM - Ned Lamont (Democratic Party and Working Families Party (fusion candidate))  
G18GOVRSTE - Bob Stefanowski (Republican Party and Independent Party (fusion candidate))  
G18GOVLHAN - Rodney Hanscomb (Libertarian Party)  
G18GOVOGRI - Oz Griebel (Griebel Frank for CT Party)  
G18GOVOGRE - Mark Stewart Greenstein (Amigo Constitution Liberty Party)  
G18GOVOWRI - Write-in Votes  

G18ATGDTON - William Tong (Democratic Party and Working Families Party (fusion candidate))  
G18ATGRHAT - Sue Hatfield (Republican Party and Independent Party (fusion candidate))  
G18ATGGGOS - Peter D. Goselin (Green Party)  

G18SOSDMER - Denise W. Merrill (Democratic Party and Working Families Party (fusion candidate))
G18SOSRCHA - Susan Chapman (Republican Party and Independent Party (fusion candidate))  
G18SOSLGWY - Heather Lynn Sylvestre Gwynn (Libertarian Party)  
G18SOSGDER - S. Michael DeRosa (Green Party)  

G18TREDWOO - Shawn Wooden (Democratic Party and Working Families Party (fusion candidate))  
G18TRERGRA - Thad Gray (Republican Party and Independent Party (fusion candidate))  
G18TRELBRO - Jesse Brohinsky (Libertarian Party)  
G18TREOWRI - Write-in Votes  

G18COMDLEM - Kevin Lembo (Democratic Party and Working Families Party (fusion candidate))  
G18COMRMIL - Kurt Miller (Republican Party and Independent Party (fusion candidate))  
G18COMLPAS - Paul Passarelli (Libertarian Party)  
G18COMGHEF - Edward G. Heflin (Green Party)  

## Election Result Processing

### Load in VEST file

In [2]:
vest_ct_18 = gp.read_file("./raw-from-source/VEST/ct_2018/ct_2018.shp")

In [3]:
data_columns = [i for i in vest_ct_18.columns if "G18" in i]

In [4]:
print(vest_ct_18.shape)

(742, 30)


In [5]:
election_results = pd.read_csv("./raw-from-source/Election_Results/ct-2018-statewide-voting-district-report.csv",skiprows=[0,1],index_col=0)

In [6]:
election_results.reset_index(inplace=True,drop=False)

In [7]:
office_list = ['Governor and Lieutenant Governor', 'United States Senator','Secretary of the State', 'Treasurer',
       'Comptroller', 'Attorney General']
election_results = election_results[election_results["OfficeName"].isin(office_list)]



In [8]:
election_results["cand_detailed"] = election_results["OfficeName"]+"-"+election_results["CandidateName"]

 G18SOSDMER - Denise W. Merrill (Democratic Party and Working Families Party (fusion candidate))
G18SOSRCHA - Susan Chapman (Republican Party and Independent Party (fusion candidate))  
G18SOSLGWY - Heather Lynn Sylvestre Gwynn (Libertarian Party)  
G18SOSGDER - S. Michael DeRosa (Green Party)   

In [9]:
cand_detailed_change_dict = {      
       'Governor and Lieutenant Governor-Ned Lamont and Susan BysiewiczMachine/Polling Place/EDRAbsenteeTotal':"G18GOVDLAM",
       'Governor and Lieutenant Governor-Bob Stefanowski and Joe MarkleyMachine/Polling Place/EDRAbsenteeTotal':"G18GOVRSTE",
       'Governor and Lieutenant Governor-Rodney Hanscomb and Jeffrey ThibeaultMachine/Polling Place/EDRAbsenteeTotal':"G18GOVLHAN",
       'Governor and Lieutenant Governor-Mark Stewart Greenstein and John DemitrusMachine/Polling Place/EDRAbsenteeTotal':"G18GOVOGRE",
       'Governor and Lieutenant Governor-Oz Griebel and Monte E FrankMachine/Polling Place/EDRAbsenteeTotal':"G18GOVOGRI",
       'Governor and Lieutenant Governor-Lee Whitnum and Jacey WyattMachine/Polling Place/EDRAbsenteeTotal':"G18GOVOWRI",
       'United States Senator-Christopher S MurphyMachine/Polling Place/EDRAbsenteeTotal':"G18USSDMUR",
       'United States Senator-*Matthew CoreyMachine/Polling Place/EDRAbsenteeTotal':"G18USSRCOR",
       'United States Senator-Richard LionMachine/Polling Place/EDRAbsenteeTotal':"G18USSLLIO",
       'United States Senator-Jeff RussellMachine/Polling Place/EDRAbsenteeTotal':"G18USSGRUS",
       'United States Senator-Kristi L TalmadgeMachine/Polling Place/EDRAbsenteeTotal':"G18USSOWRI",
       'United States Senator-Fred LinckMachine/Polling Place/EDRAbsenteeTotal':"G18USSOWRI",
       'Secretary of the State-Denise W MerrillMachine/Polling Place/EDRAbsenteeTotal':"G18SOSDMER",
       'Secretary of the State-Susan ChapmanMachine/Polling Place/EDRAbsenteeTotal':"G18SOSRCHA",
       'Secretary of the State-Heather Lynn Sylvestre GwynnMachine/Polling Place/EDRAbsenteeTotal':"G18SOSLGWY",
       'Secretary of the State-S Michael DeRosaMachine/Polling Place/EDRAbsenteeTotal':"G18SOSGDER",
       'Treasurer-*Shawn WoodenMachine/Polling Place/EDRAbsenteeTotal':"G18TREDWOO",
       'Treasurer-*Thad GrayMachine/Polling Place/EDRAbsenteeTotal':"G18TRERGRA",
       'Treasurer-Shawn WoodenMachine/Polling Place/EDRAbsenteeTotal':"G18TREDWOO",
       'Treasurer-Thad GrayMachine/Polling Place/EDRAbsenteeTotal':"G18TRERGRA",
       'Treasurer-Jesse BrohinskyMachine/Polling Place/EDRAbsenteeTotal':"G18TRELBRO",
       'Treasurer-W. Michael DownesMachine/Polling Place/EDRAbsenteeTotal':"G18TREOWRI",
       'Comptroller-Kevin LemboMachine/Polling Place/EDRAbsenteeTotal':"G18COMDLEM",
       'Comptroller-*Kurt MillerMachine/Polling Place/EDRAbsenteeTotal':"G18COMRMIL",
       'Comptroller-Kurt MillerMachine/Polling Place/EDRAbsenteeTotal':"G18COMRMIL",
       'Comptroller-Paul PassarelliMachine/Polling Place/EDRAbsenteeTotal':"G18COMLPAS",
       'Comptroller-Edward G HeflinMachine/Polling Place/EDRAbsenteeTotal':"G18COMGHEF",
       'Attorney General-*William TongMachine/Polling Place/EDRAbsenteeTotal':"G18ATGDTON",
       'Attorney General-*Sue HatfieldMachine/Polling Place/EDRAbsenteeTotal':"G18ATGRHAT",
       'Attorney General-William TongMachine/Polling Place/EDRAbsenteeTotal':"G18ATGDTON",
       'Attorney General-Sue HatfieldMachine/Polling Place/EDRAbsenteeTotal':"G18ATGRHAT",
       'Attorney General-Peter D GoselinMachine/Polling Place/EDRAbsenteeTotal':"G18ATGGGOS"
}

G18USSDMUR - Christopher S. Murphy (Democratic Party and Working Families Party (fusion candidate))  
G18USSRCOR - Matthew Corey (Republican Party)  
G18USSLLIO - Richard Lion (Libertarian Party)  
G18USSGRUS - Jeff Russell (Green Party)  
G18USSOWRI - Write-in Votes  

G18GOVDLAM - Ned Lamont (Democratic Party and Working Families Party (fusion candidate))  
G18GOVRSTE - Bob Stefanowski (Republican Party and Independent Party (fusion candidate))  
G18GOVLHAN - Rodney Hanscomb (Libertarian Party)  
G18GOVOGRI - Oz Griebel (Griebel Frank for CT Party)  
G18GOVOGRE - Mark Stewart Greenstein (Amigo Constitution Liberty Party)  
G18GOVOWRI - Write-in Votes  

G18ATGDTON - William Tong (Democratic Party and Working Families Party (fusion candidate))  
G18ATGRHAT - Sue Hatfield (Republican Party and Independent Party (fusion candidate))  
G18ATGGGOS - Peter D. Goselin (Green Party)  

G18SOSDMER - Denise W. Merrill (Democratic Party and Working Families Party (fusion candidate))
G18SOSRCHA - Susan Chapman (Republican Party and Independent Party (fusion candidate))  
G18SOSLGWY - Heather Lynn Sylvestre Gwynn (Libertarian Party)  
G18SOSGDER - S. Michael DeRosa (Green Party)  

G18TREDWOO - Shawn Wooden (Democratic Party and Working Families Party (fusion candidate))  
G18TRERGRA - Thad Gray (Republican Party and Independent Party (fusion candidate))  
G18TRELBRO - Jesse Brohinsky (Libertarian Party)  
G18TREOWRI - Write-in Votes  

G18COMDLEM - Kevin Lembo (Democratic Party and Working Families Party (fusion candidate))  
G18COMRMIL - Kurt Miller (Republican Party and Independent Party (fusion candidate))  
G18COMLPAS - Paul Passarelli (Libertarian Party)  
G18COMGHEF - Edward G. Heflin (Green Party)  

In [10]:
election_results["cand_detailed"] = election_results["cand_detailed"].map(cand_detailed_change_dict).fillna(election_results["cand_detailed"])

In [11]:
election_results

,ElectionName,TownName,ElectionCategory,Election_Date,TownClerk,OfficeName,CandidateName,PartyName,Polling_Place_Name,Machine_Count,Absentee_Count,Final_Count,cand_detailed
0,"November 2018 State Election November 06, 2018",Andover,State Election,November 06' 2018,NaN,Governor and Lieutenant Governor,Ned Lamont and Susan BysiewiczMachine/Polling ...,Democratic Party,DISTRICT 1-Andover Town Hall,593,25,618,G18GOVDLAM
1,"November 2018 State Election November 06, 2018",Andover,State Election,November 06' 2018,NaN,Governor and Lieutenant Governor,Bob Stefanowski and Joe MarkleyMachine/Polling...,Republican Party,DISTRICT 1-Andover Town Hall,832,31,863,G18GOVRSTE
2,"November 2018 State Election November 06, 2018",Andover,State Election,November 06' 2018,NaN,Governor and Lieutenant Governor,Ned Lamont and Susan BysiewiczMachine/Polling ...,Working Families Party,DISTRICT 1-Andover Town Hall,18,1,19,G18GOVDLAM
3,"November 2018 State Election November 06, 2018",Andover,State Election,November 06' 2018,NaN,Governor and Lieutenant Governor,Bob Stefanowski and Joe MarkleyMachine/Polling...,Independent Party,DISTRICT 1-Andover Town Hall,28,1,29,G18GOVRSTE
4,"November 2018 State Election November 06, 2018",Andover,State Election,November 06' 2018,NaN,Governor and Lieutenant Governor,Rodney Hanscomb and Jeffrey ThibeaultMachine/P...,Libertarian Party,DISTRICT 1-Andover Town Hall,6,0,6,G18GOVLHAN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
34593,"November 2018 State Election November 06, 2018",Woodstock,State Election,November 06' 2018,Judy Walberg,Attorney General,*William TongMachine/Polling Place/EDRAbsentee...,Democratic Party,DISTRICT 1-Woodstock Town Hall,1389,103,1492,G18ATGDTON
34594,"November 2018 State Election November 06, 2018",Woodstock,State Election,November 06' 2018,Judy Walberg,Attorney General,*Sue HatfieldMachine/Polling Place/EDRAbsentee...,Republican Party,DISTRICT 1-Woodstock Town Hall,1904,112,2016,G18ATGRHAT
34595,"November 2018 State Election November 06, 2018",Woodstock,State Election,November 06' 2018,Judy Walberg,Attorney General,William TongMachine/Polling Place/EDRAbsenteeT...,Working Families Party,DISTRICT 1-Woodstock Town Hall,52,0,52,G18ATGDTON
34596,"November 2018 State Election November 06, 2018",Woodstock,State Election,November 06' 2018,Judy Walberg,Attorney General,Sue HatfieldMachine/Polling Place/EDRAbsenteeT...,Independent Party,DISTRICT 1-Woodstock Town Hall,77,3,80,G18ATGRHAT


In [12]:
pivoted_results = pd.pivot_table(election_results,index=["Polling_Place_Name","TownName"],values=["Final_Count"],aggfunc=sum,columns="cand_detailed")

In [13]:
pivoted_results.reset_index(inplace=True,drop=False)

In [14]:
pivoted_results.columns = pivoted_results.columns.droplevel(0)


In [15]:
pivoted_results.columns

Index(['', '', 'G18ATGDTON', 'G18ATGGGOS', 'G18ATGRHAT', 'G18COMDLEM',
       'G18COMGHEF', 'G18COMLPAS', 'G18COMRMIL', 'G18GOVDLAM', 'G18GOVLHAN',
       'G18GOVOGRE', 'G18GOVOGRI', 'G18GOVOWRI', 'G18GOVRSTE', 'G18SOSDMER',
       'G18SOSGDER', 'G18SOSLGWY', 'G18SOSRCHA', 'G18TREDWOO', 'G18TRELBRO',
       'G18TREOWRI', 'G18TRERGRA', 'G18USSDMUR', 'G18USSGRUS', 'G18USSLLIO',
       'G18USSOWRI', 'G18USSRCOR'],
      dtype='object', name='cand_detailed')

In [16]:
pivoted_results.columns = ['Precinct', 'Town'  , 'G18ATGDTON', 'G18ATGGGOS', 'G18ATGRHAT', 'G18COMDLEM',
       'G18COMGHEF', 'G18COMLPAS', 'G18COMRMIL', 'G18GOVDLAM', 'G18GOVLHAN',
       'G18GOVOGRE', 'G18GOVOGRI', 'G18GOVOWRI', 'G18GOVRSTE', 'G18SOSDMER',
       'G18SOSGDER', 'G18SOSLGWY', 'G18SOSRCHA', 'G18TREDWOO', 'G18TRELBRO',
       'G18TREOWRI', 'G18TRERGRA', 'G18USSDMUR', 'G18USSGRUS', 'G18USSLLIO',
       'G18USSOWRI', 'G18USSRCOR']

In [17]:
pivoted_results = pivoted_results.fillna(0)

In [18]:
pivoted_results

,Precinct,Town,G18ATGDTON,G18ATGGGOS,G18ATGRHAT,G18COMDLEM,G18COMGHEF,G18COMLPAS,G18COMRMIL,G18GOVDLAM,G18GOVLHAN,G18GOVOGRE,G18GOVOGRI,G18GOVOWRI,G18GOVRSTE,G18SOSDMER,G18SOSGDER,G18SOSLGWY,G18SOSRCHA,G18TREDWOO,G18TRELBRO,G18TREOWRI,G18TRERGRA,G18USSDMUR,G18USSGRUS,G18USSLLIO,G18USSOWRI,G18USSRCOR
0,DISTRICT 1-1 Town Hall,Plainfield,457.0,22.0,737.0,537.0,15.0,20.0,625.0,440.0,14.0,1.0,84.0,0.0,703.0,540.0,12.0,19.0,639.0,557.0,30.0,0.0,611.0,607.0,10.0,15.0,0.0,584.0
1,DISTRICT 1-1-1a Town Hall,Plainfield,370.0,12.0,610.0,455.0,10.0,13.0,503.0,377.0,9.0,0.0,56.0,0.0,575.0,475.0,5.0,16.0,493.0,442.0,25.0,0.0,509.0,509.0,1.0,15.0,0.0,471.0
2,DISTRICT 1-1-Bd Of Ed Central Office - Cafeteria,Killingly,347.0,24.0,544.0,397.0,6.0,13.0,493.0,340.0,7.0,0.0,53.0,0.0,532.0,421.0,9.0,0.0,487.0,420.0,14.0,0.0,480.0,452.0,4.0,10.0,0.0,456.0
3,DISTRICT 1-1-Cromwell High School,Cromwell,3201.0,76.0,3440.0,3428.0,58.0,65.0,3072.0,2964.0,42.0,5.0,396.0,0.0,3473.0,3470.0,63.0,49.0,3103.0,3405.0,84.0,0.0,3136.0,3784.0,21.0,47.0,0.0,2941.0
4,DISTRICT 1-1-Cross Street School - A,Naugatuck,109.0,3.0,172.0,124.0,2.0,3.0,149.0,115.0,2.0,1.0,7.0,0.0,166.0,116.0,2.0,4.0,160.0,118.0,4.0,0.0,146.0,126.0,2.0,1.0,0.0,156.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
737,DISTRICT 9-Wesley School - District 9,Middletown,1019.0,19.0,732.0,1070.0,16.0,9.0,665.0,968.0,10.0,0.0,95.0,0.0,730.0,1091.0,18.0,12.0,647.0,1071.0,17.0,0.0,664.0,1157.0,12.0,12.0,0.0,616.0
738,DISTRICT 9-West Woods School,Hamden,1234.0,17.0,1141.0,1300.0,17.0,18.0,1032.0,1202.0,6.0,2.0,69.0,1.0,1161.0,1318.0,17.0,11.0,1031.0,1293.0,28.0,0.0,1038.0,1406.0,12.0,5.0,0.0,986.0
739,DISTRICT 9-Y W C A,Hartford,422.0,10.0,42.0,414.0,7.0,7.0,34.0,451.0,1.0,1.0,11.0,0.0,33.0,445.0,7.0,4.0,32.0,437.0,4.0,0.0,36.0,454.0,5.0,2.0,0.0,24.0
740,DISTRICT 90-1-Bunnell High School 120 21,Stratford,676.0,12.0,788.0,705.0,7.0,8.0,813.0,662.0,10.0,2.0,15.0,3.0,868.0,728.0,14.0,9.0,783.0,716.0,17.0,0.0,791.0,831.0,9.0,8.0,0.0,709.0


In [19]:
def statewide_totals_check(partner_df,source_df,column_list):
    """Compares the totals of two election result dataframes at the statewide total level

    Args:
      partner_df: DataFrame of election results we are comparing against
      source_df: DataFrame of election results we are comparing to
      column_list: List of races that there are votes for
 
    Returns:
      Nothing, only prints out an analysis
    """
    print("***Statewide Totals Check***")
    for race in column_list:
        if (partner_df[race].sum()- source_df[race].sum() != 0):
            print(race+" has a difference of "+str(partner_df[race].sum()-source_df[race].sum())+" votes")
            print("\tVEST: "+str(partner_df[race].sum())+" votes")
            print("\tSOURCES: "+str(source_df[race].sum())+" votes")
        else:
            print(race + " is equal", "\tVEST / RDH: " + str(partner_df[race].sum()))

In [20]:
statewide_totals_check(vest_ct_18,pivoted_results,data_columns)

***Statewide Totals Check***
G18USSDMUR is equal 	VEST / RDH: 825579
G18USSRCOR is equal 	VEST / RDH: 545717
G18USSLLIO has a difference of 5.0 votes
	VEST: 8838 votes
	SOURCES: 8833.0 votes
G18USSGRUS has a difference of 4.0 votes
	VEST: 6618 votes
	SOURCES: 6614.0 votes
G18USSOWRI has a difference of 5.0 votes
	VEST: 88 votes
	SOURCES: 83.0 votes
G18GOVDLAM has a difference of 1.0 votes
	VEST: 694510 votes
	SOURCES: 694509.0 votes
G18GOVRSTE has a difference of 6.0 votes
	VEST: 650138 votes
	SOURCES: 650132.0 votes
G18GOVLHAN is equal 	VEST / RDH: 6086
G18GOVOGRI is equal 	VEST / RDH: 54741
G18GOVOGRE has a difference of 17.0 votes
	VEST: 1254 votes
	SOURCES: 1237.0 votes
G18GOVOWRI has a difference of 5.0 votes
	VEST: 74 votes
	SOURCES: 69.0 votes
G18ATGDTON is equal 	VEST / RDH: 715340
G18ATGRHAT is equal 	VEST / RDH: 633360
G18ATGGGOS has a difference of 4.0 votes
	VEST: 14358 votes
	SOURCES: 14354.0 votes
G18SOSDMER is equal 	VEST / RDH: 764067
G18SOSRCHA is equal 	VEST / RDH: 58

In [21]:
def county_totals_check(partner_df,source_df,column_list,county_col,full_print=False):
    """Compares the totals of two election result dataframes at the county level

    Args:
      partner_df: DataFrame of election results we are comparing against
      source_df: DataFrame of election results we are comparing to
      column_list: List of races that there are votes for
      county_col: String of the column name that contains county information
      full_print: Boolean specifying whether to print out everything, including counties w/ similarities

    Returns:
      Nothing, only prints out an analysis
    """
    
    print("***Countywide Totals Check***")
    print("")
    diff_counties=[]
    for race in column_list:
        diff = partner_df.groupby([county_col]).sum()[race]-source_df.groupby([county_col]).sum()[race]
        for val in diff[diff != 0].index.values.tolist():
            if val not in diff_counties:
                diff_counties.append(val)
        if len(diff[diff != 0]!=0):   
            print(race + " contains differences in these counties:")
            for val in diff[diff != 0].index.values.tolist():
                county_differences = diff[diff != 0]
                print("\t"+val+" has a difference of "+str(county_differences[val])+" votes")
                print("\t\tVEST: "+str(partner_df.groupby([county_col]).sum().loc[val,race])+" votes")
                print("\t\tSOURCES: "+str(source_df.groupby([county_col]).sum().loc[val,race])+" votes")
            if (full_print):
                for val in diff[diff == 0].index.values.tolist():
                    county_similarities = diff[diff == 0]
                    print("\t"+val + ": "+ str(partner_df.groupby([county_col]).sum().loc[val,race])+" votes")
        else:
            print(race + " is equal across all counties")
            if (full_print):
                for val in diff[diff == 0].index.values.tolist():
                    county_similarities = diff[diff == 0]
                    print("\t"+val + ": "+ str(partner_df.groupby([county_col]).sum().loc[val,race])+" votes")
    if (len(diff_counties)>0):
        print()
        print(diff_counties)

In [22]:
vest_ct_18

,STATEFP20,COUNTYFP20,NAME20,G18USSDMUR,G18USSRCOR,G18USSLLIO,G18USSGRUS,G18USSOWRI,G18GOVDLAM,G18GOVRSTE,G18GOVLHAN,G18GOVOGRI,G18GOVOGRE,G18GOVOWRI,G18ATGDTON,G18ATGRHAT,G18ATGGGOS,G18SOSDMER,G18SOSRCHA,G18SOSLGWY,G18SOSGDER,G18TREDWOO,G18TRERGRA,G18TRELBRO,G18TREOWRI,G18COMDLEM,G18COMRMIL,G18COMLPAS,G18COMGHEF,geometry
0,09,011,Montville 004-00,1215,1131,20,15,1,927,1366,17,120,0,1,922,1405,35,1075,1230,29,24,1076,1228,33,1,1038,1250,27,21,"POLYGON ((-72.22762 41.51870, -72.22757 41.518..."
1,09,005,Barkhamsted 001-00,901,1046,19,9,0,726,1138,12,124,2,0,785,1148,23,854,1051,20,26,817,1085,26,0,865,1034,22,15,"POLYGON ((-73.05268 41.89076, -73.05217 41.892..."
2,09,005,Bethlehem 001-00,867,1202,18,9,0,707,1310,9,73,3,0,721,1291,17,808,1193,16,11,773,1222,18,0,802,1193,14,15,"POLYGON ((-73.26145 41.66464, -73.25791 41.664..."
3,09,005,Bridgewater 001-00,544,505,5,5,0,469,566,1,27,0,0,482,555,10,503,529,6,6,503,528,4,0,502,522,5,9,"POLYGON ((-73.40474 41.53702, -73.40457 41.537..."
4,09,005,Canaan 001-00,373,178,1,5,0,327,218,3,15,1,0,335,205,9,367,178,2,6,339,207,5,0,342,194,5,8,"POLYGON ((-73.37304 41.96030, -73.37296 41.960..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
737,09,009,Waterbury 071-02,416,295,7,3,0,350,356,6,26,2,1,353,344,8,397,290,7,7,390,296,8,1,399,285,9,6,"POLYGON ((-73.04755 41.51520, -73.04857 41.515..."
738,09,009,Waterbury 075-04,736,299,9,4,0,684,366,5,22,0,0,658,347,8,705,297,9,5,692,306,14,0,695,294,11,5,"POLYGON ((-73.04852 41.54376, -73.04838 41.543..."
739,09,009,Waterbury 071-03,1144,1119,23,13,0,999,1266,20,68,4,0,1000,1232,16,1108,1086,25,28,1077,1114,36,1,1099,1081,37,15,"POLYGON ((-73.06130 41.52992, -73.06116 41.529..."
740,09,009,New Haven 022-01,659,34,0,3,0,652,46,1,6,0,0,600,38,3,603,31,0,6,594,39,1,0,593,37,0,5,"POLYGON ((-72.93264 41.31274, -72.93213 41.312..."


In [23]:
ct_crosswalk = pd.read_csv("./raw-from-source/Crosswalk/Connecticut_Towns_-_Crosswalk_with_Tax_Codes_and_FIPS_Codes.csv")

In [24]:
ct_crosswalk = ct_crosswalk.fillna("0011")
ct_crosswalk["FIPS Code"] = ct_crosswalk["FIPS Code"].astype(str)



In [25]:
ct_crosswalk


,Tax Code,Town Name,FIPS Code
0,1,Andover,901301080.0
1,2,Ansonia,900901220.0
2,3,Ashford,901501430.0
3,4,Avon,900302060.0
4,5,Barkhamsted,900502760.0
...,...,...,...
164,165,Windsor Locks,900387070.0
165,166,Wolcott,900987560.0
166,167,Woodbridge,900987700.0
167,168,Woodbury,900587910.0


In [26]:
ct_crosswalk_dict = dict(zip(ct_crosswalk["Town Name"],ct_crosswalk["FIPS Code"].str[1:4]))

In [27]:
ct_crosswalk_dict

{'Andover': '013',
 'Ansonia': '009',
 'Ashford': '015',
 'Avon': '003',
 'Barkhamsted': '005',
 'Beacon Falls': '009',
 'Berlin': '003',
 'Bethany': '009',
 'Bethel': '001',
 'Bethlehem': '005',
 'Bloomfield': '003',
 'Bolton': '013',
 'Bozrah': '011',
 'Branford': '009',
 'Bridgeport': '001',
 'Bridgewater': '005',
 'Bristol': '003',
 'Brookfield': '001',
 'Brooklyn': '015',
 'Burlington': '003',
 'Canaan': '005',
 'Canterbury': '015',
 'Canton': '003',
 'Chaplin': '015',
 'Cheshire': '009',
 'Chester': '007',
 'Clinton': '007',
 'Colchester': '011',
 'Colebrook': '005',
 'Columbia': '013',
 'Cornwall': '005',
 'Coventry': '013',
 'Cromwell': '007',
 'Danbury': '001',
 'Darien': '001',
 'Deep River': '007',
 'Derby': '009',
 'Durham': '007',
 'Eastford': '015',
 'East Granby': '003',
 'East Haddam': '007',
 'East Hampton': '007',
 'East Hartford': '003',
 'East Haven': '009',
 'East Lyme': '011',
 'Easton': '001',
 'East Windsor': '003',
 'Ellington': '013',
 'Enfield': '003',
 'Esse

In [28]:
pivoted_results["COUNTYFP20"] =  pivoted_results["Town"].map(ct_crosswalk_dict).fillna(pivoted_results["Town"])

In [29]:
pivoted_results["COUNTYFP20"].unique()

array(['015', '007', '009', '003', '001', '013', '011', '005'],
      dtype=object)

In [30]:
county_totals_check(vest_ct_18,pivoted_results,data_columns,"COUNTYFP20",full_print=False)

***Countywide Totals Check***

G18USSDMUR is equal across all counties
G18USSRCOR is equal across all counties
G18USSLLIO contains differences in these counties:
	003 has a difference of 5.0 votes
		VEST: 2138 votes
		SOURCES: 2133.0 votes
G18USSGRUS contains differences in these counties:
	003 has a difference of 4.0 votes
		VEST: 1679 votes
		SOURCES: 1675.0 votes
G18USSOWRI contains differences in these counties:
	003 has a difference of 3.0 votes
		VEST: 28 votes
		SOURCES: 25.0 votes
	011 has a difference of 1.0 votes
		VEST: 9 votes
		SOURCES: 8.0 votes
	015 has a difference of 1.0 votes
		VEST: 7 votes
		SOURCES: 6.0 votes
G18GOVDLAM contains differences in these counties:
	009 has a difference of 1.0 votes
		VEST: 160406 votes
		SOURCES: 160405.0 votes
G18GOVRSTE contains differences in these counties:
	003 has a difference of 6.0 votes
		VEST: 144218 votes
		SOURCES: 144212.0 votes
G18GOVLHAN is equal across all counties
G18GOVOGRI is equal across all counties
G18GOVOGRE conta

## Precinct-by-Precinct

In [31]:
print(vest_ct_18["NAME20"].value_counts(dropna=False))
print(pivoted_results["Precinct"].value_counts(dropna=False))

Bridgeport 130-05    1
Madison 002-00       1
Andover 001-00       1
Goshen 001-00        1
East Haven 001-00    1
                    ..
Waterbury 074-05     1
Waterbury 075-02     1
Bridgeport 130-03    1
Trumbull 002-23      1
Waterbury 073-04     1
Name: NAME20, Length: 742, dtype: int64
DISTRICT 1-Town Hall                                13
DISTRICT 2-Korn School 1                             1
DISTRICT 4-1-Truman School                           1
DISTRICT 4-Gaylordsville Fire House                  1
DISTRICT 4-34-Middlebrook School 134                 1
                                                    ..
DISTRICT 3-1-Career High School                      1
DISTRICT 124-4-Harding High School                   1
DISTRICT 6-Mary T. Murphy School                     1
DISTRICT 2-Mansfield Fire Dept. #107@ Eagleville     1
DISTRICT 1-Cross Lane Firehouse                      1
Name: Precinct, Length: 730, dtype: int64


In [32]:
pivoted_results["NAME20"] = pivoted_results["Town"]+"-"+pivoted_results["Precinct"]
print(pivoted_results["NAME20"].value_counts(dropna=False))

Haddam-DISTRICT 2-Central Office                                         1
West Haven-DISTRICT 7-Ann V. Molloy School                               1
North Haven-DISTRICT 4-Green Acres Elementary School                     1
Coventry-DISTRICT 2-Coventry Grammer School                              1
Meriden-DISTRICT 11-Israel Putnam School                                 1
                                                                        ..
East Windsor-DISTRICT 1-Town Hall Annex                                  1
Hartford-DISTRICT 2-Liberty Christian Center-Formerly Horace Bushnell    1
Southington-DISTRICT 8-Thalberg School                                   1
Norwalk-DISTRICT 142-3-West Rocks Middle School                          1
Norwich-DISTRICT 3-Samuel Huntington Elementary School                   1
Name: NAME20, Length: 742, dtype: int64


In [33]:
join_attempt_one = pd.merge(vest_ct_18,pivoted_results,on="NAME20",how="outer",validate="1:1",indicator=True)
print(join_attempt_one["_merge"].value_counts())

join_attempt_one[join_attempt_one["_merge"]=="left_only"].to_csv("./vest.csv")
join_attempt_one[join_attempt_one["_merge"]=="right_only"].to_csv("./elections.csv")

left_only     742
right_only    742
both            0
Name: _merge, dtype: int64


In [34]:
elections_vest_id_changes = pd.read_csv("./elections_vest_id_changes.csv")

In [35]:
elections_vest_id_changes_dict = dict(zip(elections_vest_id_changes["election_ID"],elections_vest_id_changes["vest_ID"]))

In [36]:
new_elections_vest_id_changes_dict = {}
for val in elections_vest_id_changes_dict:
    if val in list(pivoted_results["NAME20"].unique()):
        new_elections_vest_id_changes_dict[val]=elections_vest_id_changes_dict[val]

In [37]:
pivoted_results["NAME20"] = pivoted_results["NAME20"].map(new_elections_vest_id_changes_dict).fillna(pivoted_results["NAME20"])

In [38]:
join_attempt_two = pd.merge(vest_ct_18,pivoted_results,on="NAME20",how="outer",validate="1:1",indicator=True)
print(join_attempt_two["_merge"].value_counts())

join_attempt_two[join_attempt_two["_merge"]=="left_only"].to_csv("./vest.csv")
join_attempt_two[join_attempt_two["_merge"]=="right_only"].to_csv("./elections.csv")

both          742
left_only       0
right_only      0
Name: _merge, dtype: int64


In [39]:
def precinct_votes_check(merged_df,column_list,vest_on_left,name_col,print_level=0):
    """Checks a merged dataframe with two election results at the precinct level

    Args:
      merged_df: DataFrame with one set of election results joined to another
      column_list: List of races that there are votes for
      vest_on_left: Boolean specifying whether VEST data is on the left side of merged_df
      name_col: String of the column name to refer to precincts when a difference occurs
      print_level: Integer that specifies how large the vote difference in a precinct must be to be printed

    Returns:
      Nothing, only prints out an analysis
    """
    merged_df = merged_df.sort_values(by=[name_col],inplace=False)
    matching_rows = 0
    different_rows = 0
    diff_list=[]
    diff_values = []
    max_diff = 0
    for index,row in merged_df.iterrows():
        same = True
        for i in column_list:
            left_data = i + "_x"
            right_data = i + "_y"
            if ((row[left_data] is None) or (row[right_data] is None) or (np.isnan(row[right_data])or(np.isnan(row[left_data])))):
                print("FIX NaN value at: ", row[name_col])
                return;
            diff = abs(row[left_data]-row[right_data])
            if (diff>0):
                same = False
                diff_values.append(abs(diff))
                if (diff>max_diff):
                    max_diff = diff
            if(diff>print_level):
                if (vest_on_left):
                    print(i, "{:.>72}".format(row[name_col]), "(V)","{:.>5}".format(int(row[left_data]))," (S){:.>5}".format(int(row[right_data])),"(D):{:>5}".format(int(row[left_data]-row[right_data])))                           
                else:
                    print(i, "{:.>72}".format(row[name_col]), "(S)","{:.>5}".format(int(row[left_data]))," (V){:.>5}".format(int(row[right_data])),"(D):{:>5}".format(int(row[left_data]-row[right_data])))
        if(same != True):
            different_rows +=1
            diff_list.append(row[name_col])
        else:
            matching_rows +=1
    print("")
    print("There are ", len(merged_df.index)," total rows")
    print(different_rows," of these rows have election result differences")
    print(matching_rows," of these rows are the same")
    print("")
    print("The max difference between any one shared column in a row is: ", max_diff)
    if(len(diff_values)!=0):
        print("The average difference is: ", str(sum(diff_values)/len(diff_values)))
    count_big_diff = len([i for i in diff_values if i > 10])
    print("There are ", str(count_big_diff), "precinct results with a difference greater than 10")
    print("")
    print("All precincts containing differences:")
    diff_list.sort()
    print(diff_list)

In [40]:
precinct_votes_check(join_attempt_two[join_attempt_two["_merge"]=="both"],data_columns,True,"NAME20",print_level=0)

G18GOVOGRE ...........................................................Berlin 001-00 (V) ....2  (S)....0 (D):    2
G18GOVOWRI ..........................................................Bethany 001-00 (V) ....1  (S)....0 (D):    1
G18GOVDLAM .......................................................East Haven 001-03 (V) ...64  (S)...63 (D):    1
G18TREOWRI ......................................................Glastonbury 001-00 (V) ....1  (S)....0 (D):    1
G18USSLLIO ......................................................Glastonbury 002-00 (V) ....5  (S)....0 (D):    5
G18USSGRUS ......................................................Glastonbury 002-00 (V) ....4  (S)....0 (D):    4
G18USSOWRI ......................................................Glastonbury 002-00 (V) ....1  (S)....0 (D):    1
G18GOVRSTE ......................................................Glastonbury 002-00 (V) ..135  (S)..129 (D):    6
G18GOVOGRE ......................................................Glastonbury 002-00 (V) 